In [13]:
#Imports
#Loading our data
import csv as csv
import cv2
import numpy as np

#Keras imports
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, MaxPooling2D, Activation
from keras import backend as K

### Loading the data

In order to load our data we firstly read the driving_log.csv file which contains the paths to the left, center and right cameras and also info about steering angle, throttle, break and speed. 

We will want to automate the loading process in order to easily load our own data which is stored in different directories. For that we used several libraries such as csv (read csv files) and cv2 (read images).

In [2]:
#Reading the files

#Reading the csv file
def readCSV(in_path):
    lines = []
    with open(in_path) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) #Skip header
        for line in reader:
            lines.append(line)
            
    return lines

def readData(in_dir, in_lines):
    images = []
    measurements = []
    for line in in_lines:
        source_path = line[0]
        filename = source_path.split('/')[-1]
        path = in_dir + filename
        image = cv2.imread(path)
        images.append(image)
        measurement = float(line[3])
        measurements.append(measurement)
    
    return np.array(images), np.array(measurements)

In [3]:
imgs_path = './data/IMG/'
csv_path = './data/driving_log.csv'

X_train, y_train = readData(imgs_path, readCSV(csv_path))

print(type(y_train))

print('Data loaded successfully!')

<class 'numpy.ndarray'>
Data loaded successfully!


In [4]:
print(type(y_train[0]))

<class 'numpy.float64'>


### Basic neural network using Keras

We will implement a basic neural network in order to verify that everything is working before implementing a more complex model. This network will just going to be a flattened image connected to a single output node. This single output node will predict the stearing angle, thus converting this model into a regression network. In contrast with a classification network, we may apply a softmax activation function to the output layer. Nevertheless in this case we will not use an activation function. We will directly predict the steering measurement. 

For this basic implementation we will use Keras as a library which works with tensorflow as backend. This will simplify our implementation and will be great for prototyping. Let's go ahead!

Improvement 1: In order to improve our model we need to preprocess our input data. For that we will add two preprocessing steps: normalization and  mean centering the data. We will add a lambda layer to our model. After doing this, we can decrease the training epochs a lot. We will fix the number of epochs in 2.

In [7]:
#Model definition

model = Sequential()
#Lambda layer for normalizing our data. In order to mean center the data, we will
#need to substract -0.5 (shifting the model down) to the normalized data. 
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

#Model compilation
#For the loss function we will use Mean Squared Error (MSE). We will minimize the 
#error between the steering measurement which the network predicts and the ground 
#truth steering measurements provided by the dataset
model.compile(loss='mse', optimizer='adam')
#we also shuffle the data and split off 20% of the data to use for a validation set
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs= 2)

#Keras by default will run 10 epochs. Nevertheless with 10 epochs we will 
#overfit the training data. For that reason we will only perform 6 epochs
model.save('basic_model.h5')

K.clear_session()

Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 2s 384us/step - loss: 1.8318 - val_loss: 0.7415
Epoch 2/2
6428/6428 [==============================] - 2s 370us/step - loss: 2.9713 - val_loss: 2.7356


#### We will try to autonomous drive the car with this model

### Trying a more complex network such as LeNet-5 architecture

We will implement LeNet-5 architecture using Keras.

In [15]:
model = Sequential()
#Lambda layer for normalizing our data. In order to mean center the data, we will
#need to substract -0.5 (shifting the model down) to the normalized data. 
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape))

#First set of CONV => RELU => POOL
model.add(Conv2D(3, (5, 5), input_shape= X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))

#Second set of CONV => RELU => POOL
model.add(Conv2D(6, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))

#Setting the FCs layers
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))

#Output layer
model.add(Dense(1))

#Model compilation
#For the loss function we will use Mean Squared Error (MSE). We will minimize the 
#error between the steering measurement which the network predicts and the ground 
#truth steering measurements provided by the dataset
model.compile(loss='mse', optimizer='adam')
#we also shuffle the data and split off 20% of the data to use for a validation set
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs= 2)

#Keras by default will run 10 epochs. Nevertheless with 10 epochs we will 
#overfit the training data. For that reason we will only perform 6 epochs
model.save('lenet_model.h5')

K.clear_session()

Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 4s 672us/step - loss: 1.0497 - val_loss: 0.0153
Epoch 2/2
6428/6428 [==============================] - 4s 643us/step - loss: 0.0124 - val_loss: 0.0131
